# Convolutionnel Neural Networks For Facial Expression Recognition 
****
Emotion detection from facial expression is one of the most active research fields, and plays a huge part in today’s technology. It can be implemented using machine learning algorithms, although these can’t provide  a hundred percent accurate solution since facial expression are not always the same and they depend on the person, the brightness, the position, and so on. This Notebook, presents an implementation of a deep learning algorithm for emotion detection using Convolutional Neural Network on a preprocessed images from a dataset FER. The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).
Our choice of using CNN for this matter is based on the fact that this algorithm performs better than other solutions. Also, to conduct this experiment we have used a dataset which is a mix of other datasets like JAFFE and that was provided by Kaggle in the context of a competition.

****
This work was made by:
    * Nasr Abdelhamid 
                        abdelnasr7@gmail.com
    * Omar Harchich 
                        omar.harchich@gmail.com
Supervised by:
    * Professor Elhannani Assmaa.
    * kjhk Fatima Zahra Salmam.


In [ ]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

%matplotlib inline

In [ ]:
df = pd.read_csv('../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')

In [ ]:
ls ../input/facial-expression-recognitionferchallenge/fer2013/fer2013

In [ ]:
df.head()

In [ ]:
ls ../input/facial-expression-recognitionferchallenge/fer2013/fer2013

In [ ]:
df["Usage"].value_counts()

In [ ]:
train = df[["emotion", "pixels"]][df["Usage"] == "Training"]
train.isnull().sum()

In [ ]:
train['pixels'] = train['pixels'].apply(lambda im: np.fromstring(im, sep=' '))
x_train = np.vstack(train['pixels'].values)
y_train = np.array(train["emotion"])
x_train.shape, y_train.shape

In [ ]:
public_test_df = df[["emotion", "pixels"]][df["Usage"]=="PublicTest"]

In [ ]:
public_test_df["pixels"] = public_test_df["pixels"].apply(lambda im: np.fromstring(im, sep=' '))
x_val = np.vstack(public_test_df["pixels"].values)
y_val = np.array(public_test_df["emotion"])

In [ ]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_val = x_val.reshape(-1, 48, 48, 1)
x_train.shape, x_val.shape

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
y_train.shape, y_val.shape

In [ ]:
import seaborn as sns
plt.figure(0, figsize=(12,6))
for i in range(1, 13):
    plt.subplot(3,4,i)
    plt.imshow(x_train[i, :, :, 0], cmap="gray")

plt.tight_layout()
plt.show()

# Build The Model

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [ ]:
def buildModel(width, height, depth):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		chanDim = -1
		# CONV => RELU => POOL
		model.add(Conv2D(64, 3, data_format="channels_last", kernel_initializer="he_normal", 
                 input_shape=(48, 48, 1)))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.25))

		# softmax classifier
		model.add(Dense(7))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [ ]:
# batch size, and image dimensions
EPOCHS = 100
INIT_LR = 1e-3
BS = 32

In [ ]:
model = buildModel(width=48, height=48,depth=1)

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam


In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
# run model
hist = model.fit_generator( aug.flow(x_train, y_train, batch_size=BS), epochs=EPOCHS,
                 shuffle=True,
                 steps_per_epoch=len(x_train) // BS,
                 validation_data=(x_val, y_val),
                 verbose=1)

# save model to json
model_json = model.to_json()
with open("face_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
plt.figure(figsize=(14,3))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(hist.history['loss'], color='b', label='Training Loss')
plt.plot(hist.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(hist.history['acc'], color='b', label='Training Accuracy')
plt.plot(hist.history['val_acc'], color='r', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import os
image_test = "../input/image-test/test/test"
os.mkdir('test_pretraitement')

In [ ]:
import glob
import cv2 as cv
from keras.preprocessing.image import img_to_array
test_pretraitement = 'test_pretraitement'

In [ ]:
data_test = {}
labels_test = {}
face_cascade = cv.CascadeClassifier('../input/haarcascade/haarcascade_frontalface_default.xml')
i = 0
for img in glob.glob(image_test+"/*.jpg"):
    image = cv.imread(img)
    name = img.split('/')[-1]
    
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) # convert to greyscale
    height, width = image.shape[:2]
    faces = face_cascade.detectMultiScale(gray_image, 1.3, 1)
    if isinstance(faces, tuple):
        resized_image = cv.resize(gray_image, (48, 48))
        cv.imwrite(test_pretraitement+'/'+name,resized_image)
    #print(faces)
    elif isinstance(faces, np.ndarray):
        for (x,y,w,h) in faces:
            if w * h < (height * width) / 3:
                resized_image = cv.resize(gray_image, (48, 48)) 
                cv.imwrite(test_pretraitement+'/'+name,resized_image)
            else:
                
                #cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                roi_gray = gray_image[y:y+h, x:x+w]
                #print(len(roi_gray))
                resized_image = cv.resize(roi_gray, (48, 48))
                cv.imwrite(test_pretraitement+'/'+name, resized_image)
    image = resized_image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    data_test[name] = image
    #data.append(img_to_array(resized_image))

In [ ]:
data_predict = {}
for key,value in data_test.items():
    predict = model.predict(value)
    idx = np.argmax(predict)
    #l= lb.classes_[idx]
    data_predict[key] = idx
    #print(idx)

In [ ]:
final_data = pd.DataFrame(list(data_predict.items()),
                      columns=['Image','Emotion'])

In [ ]:
#pd.read_csv('submissionsref.csv')
mapping_emotion = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 6:'neutral', 4:'sadness', 5:'surprise'}
final_data['Emotion'] = final_data['Emotion'].map(mapping_emotion)

In [ ]:
final_data.to_csv('submissionsref.csv')

In [ ]:
rd_df = pd.read_csv('submissionsref.csv')

In [ ]:
rd_df.shape